In [ ]:
!pip install transformers[all]

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers[sentencepiece]


In [ ]:
ch_text = '//// <!-- hello 我叫萨拉，我住在伦敦。--> //'

In [ ]:
!pip install 'pandas == 1.3.5'

In [130]:
replace_text('第三步，md5(第二步结果)')

'Step 3, md5 (step 2 outcome)'

## Code


In [ ]:
!pip install python-magic

In [ ]:
!sudo apt-get install libmagic1

In [ ]:
import subprocess as sub

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
#import magic
#import shlex
import os
import re
import pickle
import fileinput
import sys
import xml.etree.ElementTree as ET
import pandas as pd
from bs4 import BeautifulSoup


### Load model once and use it  

In [ ]:
zh2en_model = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-zh-en')
zh2en_tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-zh-en')

saved_model = pickle.dumps(zh2en_model)
saved_tokenizer = pickle.dumps(zh2en_tokenizer)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/786k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/788k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

## Model Cells

### Translate chinese text using saved model 

In [ ]:
def translate_text(ch_text):
  """
  pass your chinese text and it returns its english translation 
  """

  zh2en_model = pickle.loads(saved_model)
  zh2en_tokenizer = pickle.loads(saved_tokenizer)
  zh2en_translation = pipeline('translation_zh_to_en', 
                      model=zh2en_model, 
                      tokenizer=zh2en_tokenizer)
  return zh2en_translation(ch_text)

In [ ]:
translate_text(ch_text)

[{'translation_text': "/// <! > - Hello, I'm Sarah, I live in London."}]

### get type of file to detect its comment character (NOT USED)

*will be better if it detects file type by its content not its name and extension*

In [ ]:
def get_type_of_file(data_path):
  """
  pass file path with its .format otherwise it faces some problems
  """
  try:
    #return magic.Magic().from_file(data_path)   
    #
    # p = sub.Popen(data_path , stdout=sub.PIPE, stderr=sub.PIPE)
    # output, errors = p.communicate()
    #
    # filename = data_path
    # cmd = shlex.split('file --mime-type {0}'.format(filename))
    # result = sub.check_output(cmd)
    # mime_type = result.split()[-1]
    # return mime_type
    file_name, file_extension = os.path.splitext(data_path)
    return file_extension
  except IOError:
    return 'No File Found'

In [ ]:
def get_index_of_chinese_text(file):
  pass

### Define a dictionary of file type and its comment character

In [85]:
comment_format_start = {'html':['<!--'],'htm':['<!--'],'xml':['<!--'],'js':['//','/*'],'java':['/*','//'],'py':['#'],
                        'vue':['<!--','/*','//'],'css':['/*','//'],'scss':['/*','//']}
comment_format_end = {'html':'-->','htm':'-->','xml':'-->','js':'*/','vue':['-->','*/'],'css':'*/','scss':'*/','java':'*/'}


### Searches for chinese characters and returns a modified translated same text 

In [ ]:
def replace_text(line):
  """
  pass line you want to change and the text before to replace new line 
  in new text
  """

  if re.search('[\u4e00-\u9fff]',line) != None:
    #line = line.replace('Hello', 'import')
    #sys.stdout.write(line)

    changes = line.replace(line, translate_text(line)[0]['translation_text'])
    return changes 

  else:
    return line 

### Parse Python Files

In [ ]:
def parse_python(f,replacement):
  closed = True
  for line in f.readlines():
    #print(line)
    if closed:
    
      if line.strip().startswith('#') :
        replacement.append("#" + replace_text(line.split('#')[1])) 

      if len(line.strip().split('#'))>1 and line.strip().split('#')[0] != "" :
        replacement.append(line.strip().split('#')[0]+"{}".format('#')+replace_text(line.strip().split('#')[1]))

      if line.strip().startswith('"""') :
        closed = False
        if line.strip().endswith('"""') and len(line.split('"""'))>2:
          #print(line)
          #replacement.append('"""' +replace_text( line.strip().split('"""')[1] )+ '"""')
          replacement.append(re.findall("^ *",line)[0]+'"""\n' +re.findall("^ *",line)[0] + replace_text( line.strip().split('"""')[1] )+ 
                             '\n'+re.findall("^ *",line)[0] + '"""\n')
          #print('here')
          closed = True
        else:
          replacement.append(line)
        
      #
      
      #re.findall("""(['"])(?:(?=(\\?))\2([\u4e00-\u9fff\d\s\W]))*?\1""",'df_arts["考室30---+-号56451"    " 考 2 zzz  135 "] = "号abc"')

      #if line.strip().startswith('\'') :
        #replacement.append('\'' + replace_text(line.split('\'')[1]))
        #while line.find('\'') == -1:
         # replacement.append(replace_text(line.split('\'')[1]))

        #replacement.append( replace_text(line.split('\'')[1])+'\'' )

        
      #if line.strip().startswith('\"') :
        #replacement.append('\"' + replace_text(line.split('\"')[1]))
        #while line.find('\"') == -1:
         # replacement.append(replace_text(line.split('\"')[1]))

        #replacement.append( replace_text(line.split('\"')[1])+'\"' )

      else:
        replacement.append(line)

    else:
      #should i add this?
      #replacement.append( replace_text(line.split('"""')[1])+'"""' )
      #print('Im here')
      replacement.append(replace_text(line)+'\n')
      if line.strip().startswith('"""') or line.strip().endswith('"""'):
        #print('im here')
        closed = True

  return replacement


### Parse XML

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
def parse_xml(f,replacement):
  # tree = ET.parse(f)
  # root = tree.getroot()
  # newsitems = []


  #df = pd.read_xml(f, xpath="./context/message/source")
  #df
  
    # iterate news items
  # for item in root.findall('./context/message/source'):
  #   #print(item)
  #   for child in item:
  #     print(child)

  #     # empty news dictionary
  #     news = {}

  #     # iterate child elements of item
  #     for child in item:

  #         # special checking for namespace object content:media
  #         if child.tag == '{http://search.yahoo.com/mrss/}content':
  #             news['media'] = child.attrib['url']
  #         else:
  #             news[child.tag] = child.text.encode('utf8')

  #     # append news dictionary to news items list
  #     newsitems.append(news)
    
  # # return news items list
  # return newsitems

  for line in f.readlines():
    if line.strip().startswith('<source>'):
      #print('hello')
      replacement.append(line)
      replacement.append('<translation type="unfinished">' + translate_text(line.strip()[8:line.strip().find('</source>')+1])[0]['translation_text'] + '</translation>\n')

    else:    
      replacement.append(line)

  return replacement

### Parse Vue

In [106]:
def parse_vue(f,replacement):

  comment_format_e = comment_format_end['vue']
  comment_format_s = comment_format_start['vue']

#flag to check if a multiple line comment is done or not
  comment_closed = True
  
  #flag to check if line is the first line of file
  first_line = True

    #read file line by line
  for line in f.readlines():
  #for mark,line in enumerate(f.readlines()):
    #print(line)

    #checks if it's in the multiple line comment, 
    #if not, one line comment cases should be checked
    if comment_closed:
      
      # for file types containing different comment characters
      for i in comment_format_s:
        count = 0
        #if the comment started from first of line
        if line.strip().startswith(i):
          #replacement.append("{}".format(i) + replace_text(line.strip().split(i)[1]))
                    
          replacement.append(re.findall("^ *",line)[0] + i + replace_text(line.strip().split(i)[1])+'\n'
                              # +re.findall("^ *",line)[0]
                              )
          #print('startswith{}'.format(i),mark)
          #try:

          #if the line hasn't been ended with ending comment character, next lines should be considered as comment
          if not line.endswith(comment_format_e[count]) and len(comment_format_s)>1 and file_type not in ['html','htm','vue']:
            if i == comment_format_s[0] or i == comment_format_s[1]:
              comment_closed = False
          #except NameError:

          if not line.strip().endswith(comment_format_e[count]) and file_type in ['html','htm','vue']:
            # if i == comment_format_s[0]:
            #print('daali',line)
            comment_closed = False

          # else:
          #   print('one line comment')

        #if comment started from middle of line
        if len(line.strip().split(i))>1 and line.strip().split(i)[0] != "" :
          replacement.append(re.findall("^ *",line)[0]+ line.strip().split(i)[0]+"{}".format(i)+
                              replace_text(line.strip().split(i)[1]+'\n'
                            #  +re.findall("^ *",line)[0]
                            ) )
          #print('contains one line comment',mark)


        #else:
      #if it's first line translate it by the way
        if first_line:
          if line.strip().startswith('<!--') or not line.strip().startswith(comment_format_s[0]) :
            #print(line)
            replacement.append('')
          else:
            #print(line, 'hello')
            if len(line.strip().split(comment_format_s[0]))>1  :
              replacement.append(replace_text(line.split(comment_format_s[0])[1]))
            if len(line.strip().split(comment_format_s[1]))>1 :
              replacement.append(replace_text(line.split(comment_format_s[1])[1]))
            else:
              replacement.append(replace_text(line))
        count +=1
      #append the line to new text if it's not repeated   
    if len(replacement)>0:
      #print(mark)
      if replacement[-1] != line[:line.index('\n')]:
        replacement.append(line)
        #print('no comment found')

        

      #We are in a multiple line comment, so it should translate it all till it sees ending comment character
      else:
        replacement.append(replace_text(line))
        #print('comment continues',mark)

      for i in comment_format_e:
        if line.find(i):
          comment_closed = True

      first_line = False


  return replacement

### Parse Text

In [114]:
def parse_text(file_type,comment_format_s,f,replacement):

  if file_type == 'py':
      return parse_python(f,replacement)

  if file_type == 'ts' or file_type == 'xml':
    return parse_xml(f,replacement)

  if file_type == 'vue':
    #print('im vue')
    return parse_vue(f,replacement)

  else:
    #flag to check if a multiple line comment is done or not
    comment_closed = True
    
    #flag to check if line is the first line of file
    first_line = True

    if comment_format_end.get(file_type) != None :
      comment_format_e = comment_format_end[file_type]

    if comment_format_s != None:
      #read file line by line
      for line in f.readlines():
      #for mark,line in enumerate(f.readlines()):
        #print(line)

        #checks if it's in the multiple line comment, 
        #if not, one line comment cases should be checked
        if comment_closed:
          
          # for file types containing different comment characters
          for i in comment_format_s:

            #if the comment started from first of line
            if line.strip().startswith(i):
              #replacement.append("{}".format(i) + replace_text(line.strip().split(i)[1]))
                       
              replacement.append(re.findall("^ *",line)[0] + i + replace_text(line.strip().split(i)[1])+'\n'
                                  +re.findall("^ *",line)[0]
                                 )
              #print('startswith{}'.format(i),mark)
              #try:

              #if the line hasn't been ended with ending comment character, next lines should be considered as comment
              if not line.endswith(comment_format_e) and len(comment_format_s)>1 and file_type not in ['html','htm','vue']:
                if i == comment_format_s[0]:
                  comment_closed = False
              #except NameError:

              if not line.strip().endswith(comment_format_e) and file_type in ['html','htm','vue']:
                # if i == comment_format_s[0]:
                print('daali',line)
                comment_closed = False

              # else:
              #   print('one line comment')

            #if comment started from middle of line
            if len(line.strip().split(i))>1 and line.strip().split(i)[0] != "" :
              replacement.append(re.findall("^ *",line)[0]+ line.strip().split(i)[0]+"{}".format(i)+
                                 replace_text(line.strip().split(i)[1]+'\n'
                                #  +re.findall("^ *",line)[0]
                                ) )
              #print('contains one line comment',mark)


            #else:
          #if it's first line translate it by the way
          if first_line:
            if line.strip().startswith('package') or not line.strip().startswith(comment_format_s[0]) :
              #print(line)
              replacement.append('')
            else:
              #print(line, 'hello')
              if len(line.strip().split(comment_format_s[0]))>1  :
                replacement.append(replace_text(line.split(comment_format_s[0])[1]))
              else:
                replacement.append(replace_text(line))

          #append the line to new text if it's not repeated   
          if len(replacement)>0:
            #print(mark)
            if replacement[-1] != line[:line.index('\n')]:
              replacement.append(line)
              #print('no comment found')

        #We are in a multiple line comment, so it should translate it all till it sees ending comment character
        else:
          replacement.append(re.findall("^ *",line)[0] + replace_text(line)+re.findall("^ *",line)[0]+'\n')
          #print('comment continues',mark)

          if line.find(comment_format_e):
            comment_closed = True

        first_line = False
    #except NameError:

    #file doesn't contain comments and we should search for chinese characters whole th file
    else:
      #print('This file doesnt contain comment')
      for line in f.readlines():
        replacement.append(replace_text(line))

    return replacement


### Main process of parsing text and creating new text file

In [107]:

def process_text(input_path,path):
  f = open(input_path,'a')
  f.write('\n\n\n')
  f.close()
  try:
    f = open(input_path,'r')
    #get type of file based on its name (name.type)
    file_name, file_type = os.path.splitext(input_path)
    file_type = file_type[1:].lower()

    #new text file to be written
    replacement = []

    #try:

    #if the file type contains comment or not (like .md or .txt files)
    #if the file is comment included, checks all possible comment formats 
    if comment_format_start.get(file_type) != None:
      comment_format_s = comment_format_start[file_type]
      #print(comment_format_s)
    else:
      comment_format_s =None

    #searches for ending comment character, if the file type supports
    if comment_format_end.get(file_type) != None :
      comment_format_e = comment_format_end[file_type]

    replacement = parse_text(file_type,comment_format_s,f,replacement)

    f.close()
    #write new text in a translated file
    if len(replacement) != 0:
      fout = open("{}/{}_.{}".format(path,file_name,file_type), "w")
      fout.writelines(replacement)
      fout.close()  

  except IOError:
    print('No File Found')



### Run this! 

In [115]:
process_text('./output_folder2/business/ReportApplication.java','./')

In [ ]:
!unzip "src.zip" -d "./output_folder4/"

In [ ]:
!unzip "business.zip" -d "./output_folder2/"

In [ ]:
import os

yourpath = './output_folder2'

for root, dirs, files in os.walk(yourpath, topdown=False):
    for name in files:
      file_name, file_type = os.path.splitext(name)
      file_type = file_type[1:].lower()
      if file_type in list(comment_format_start.keys()) or file_type=='json':
        print(os.path.join(root, name))
        process_text(os.path.join(root, name),'./')
        
        #print('daali')
      #else:
        #print(os.path.join(root, name))
        #process_text(os.path.join(root, name),'./')
    # for name in dirs:
    #     print(os.path.join(root, name))
        

In [ ]:
!zip -r /content/output_folder2/bus_trans.zip /content/output_folder2

In [ ]:
for filename in os.listdir('./test'):
  process_text('./test/{}'.format(filename))

In [ ]:
df = pd.read_xml('WorkPro_fa.ts',xpath='./context/message/source')

In [ ]:
f = open('WorkPro_fa.ts','r')
replacement = []

In [ ]:
for line in f.readlines():
  if line.strip().startswith('<source>'):
    print('hello')
    replacement.append(line)
    replacement.append('<translation type="unfinished">' + translate_text(line.strip()[8:line.strip().find('</source>')+1])[0]['translation_text'] + '</translation>\n')

  else:    
    replacement.append(line)
f.close()

In [ ]:
replacement

---

## Script

In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-
def translate_text(ch_text):
  """
  pass your chinese text and it returns its english translation 
  """

  zh2en_model = pickle.loads(saved_model)
  zh2en_tokenizer = pickle.loads(saved_tokenizer)
  zh2en_translation = pipeline('translation_zh_to_en', 
                      model=zh2en_model, 
                      tokenizer=zh2en_tokenizer)
  return zh2en_translation(ch_text)

comment_format_start = {'html':['<!--'],'htm':['<!--'],'xml':['<!--'],'js':['//','/*'],'java':['//','/*'],'py':['#']}
comment_format_end = {'html':'-->','htm':'-->','xml':'-->','js':'*/','java':'*/'}

def replace_text(line):
  """
  pass line you want to change and the text before to replace new line 
  in new text
  """

  if re.search('[\u4e00-\u9fff]',line) != None:
    #line = line.replace('Hello', 'import')
    #sys.stdout.write(line)

    changes = line.replace(line, translate_text(line)[0]['translation_text'])
    return changes 

  else:
    return line 

def parse_python(f,replacement):
  closed = True
  for line in f.readlines():
    #print(line)
    if closed:
    
      if line.strip().startswith('#') :
        replacement.append("#" + replace_text(line.split('#')[1])) 

      if len(line.strip().split('#'))>1 and line.strip().split('#')[0] != "" :
        replacement.append(line.strip().split('#')[0]+"{}".format('#')+replace_text(line.strip().split('#')[1]))

      if line.strip().startswith('"""') :
        closed = False
        if line.strip().endswith('"""') and len(line.split('"""'))>2:
          #print(line)
          #replacement.append('"""' +replace_text( line.strip().split('"""')[1] )+ '"""')
          replacement.append(re.findall("^ *",line)[0]+'"""\n' +re.findall("^ *",line)[0] + replace_text( line.strip().split('"""')[1] )+ 
                             '\n'+re.findall("^ *",line)[0] + '"""\n')
          #print('here')
          closed = True
        else:
          replacement.append(line)
        
      #
      
      #re.findall("""(['"])(?:(?=(\\?))\2([\u4e00-\u9fff\d\s\W]))*?\1""",'df_arts["考室30---+-号56451"    " 考 2 zzz  135 "] = "号abc"')

      #if line.strip().startswith('\'') :
        #replacement.append('\'' + replace_text(line.split('\'')[1]))
        #while line.find('\'') == -1:
         # replacement.append(replace_text(line.split('\'')[1]))

        #replacement.append( replace_text(line.split('\'')[1])+'\'' )

        
      #if line.strip().startswith('\"') :
        #replacement.append('\"' + replace_text(line.split('\"')[1]))
        #while line.find('\"') == -1:
         # replacement.append(replace_text(line.split('\"')[1]))

        #replacement.append( replace_text(line.split('\"')[1])+'\"' )

      else:
        replacement.append(line)

    else:
      #should i add this?
      #replacement.append( replace_text(line.split('"""')[1])+'"""' )
      #print('Im here')
      replacement.append(replace_text(line)+'\n')
      if line.strip().startswith('"""') or line.strip().endswith('"""'):
        #print('im here')
        closed = True

  return replacement

def parse_text(file_type,comment_format_s,f,replacement):

  if file_type == 'py':
      return parse_python(f,replacement)

  else:
    #flag to check if a multiple line comment is done or not
    comment_closed = True
    
    #flag to check if line is the first line of file
    first_line = True

    if comment_format_end.get(file_type) != None :
      comment_format_e = comment_format_end[file_type]

    if comment_format_s != None:
      #read file line by line
      for line in f.readlines():
      #for mark,line in enumerate(f.readlines()):
        #print(line)

        #checks if it's in the multiple line comment, 
        #if not, one line comment cases should be checked
        if comment_closed:
          
          # for file types containing different comment characters
          for i in comment_format_s:

            #if the comment started from first of line
            if line.strip().startswith(i):
              #replacement.append("{}".format(i) + replace_text(line.strip().split(i)[1]))
              replacement.append(i + replace_text(line.strip().split(i)[1]))
              #print('startswith{}'.format(i),mark)
              #try:

              #if the line hasn't been ended with ending comment character, next lines should be considered as comment
              if not line.endswith(comment_format_e) and len(comment_format_s)>1:
                if i == comment_format_s[1]:
                  comment_closed = False
              #except NameError:
              else:
                print('one line comment')

            #if comment started from middle of line
            if len(line.strip().split(i))>1 and line.strip().split(i)[0] != "" :
              replacement.append(line.strip().split(i)[0]+"{}".format(i)+replace_text(line.strip().split(i)[1]))
              #print('contains one line comment',mark)


            #else:
          #if it's first line translate it by the way
          if first_line:
            if len(line.split(comment_format_s[1]))>1:
              replacement.append(replace_text(line.split(comment_format_s[1])[1]))
            else:
              replacement.append(replace_text(line))

          #append the line to new text if it's not repeated   
          if len(replacement)>0:
            #print(mark)
            if replacement[-1] != line[:line.index('\n')]:
              replacement.append(line)
              #print('no comment found')

        #We are in a multiple line comment, so it should translate it all till it sees ending comment character
        else:
          replacement.append(replace_text(line))
          #print('comment continues',mark)

          if line.find(comment_format_e):
            comment_closed = True

        first_line = False
    #except NameError:

    #file doesn't contain comments and we should search for chinese characters whole th file
    else:
      #print('This file doesnt contain comment')
      for line in f.readlines():
        replacement.append(replace_text(line))

    return replacement


def process_text(input_path):
  f = open(input_path,'a')
  f.write('\n\n\n')
  f.close()
  try:
    f = open(input_path,'r')
    #get type of file based on its name (name.type)
    file_name, file_type = os.path.splitext(input_path)
    file_type = file_type[1:].lower()

    #new text file to be written
    replacement = []

    #try:

    #if the file type contains comment or not (like .md or .txt files)
    #if the file is comment included, checks all possible comment formats 
    if comment_format_start.get(file_type) != None:
      comment_format_s = comment_format_start[file_type]
      #print(comment_format_s)
    else:
      comment_format_s =None

    #searches for ending comment character, if the file type supports
    if comment_format_end.get(file_type) != None :
      comment_format_e = comment_format_end[file_type]

    replacement = parse_text(file_type,comment_format_s,f,replacement)

    f.close()
    #write new text in a translated file
    if len(replacement) != 0:
      fout = open("{}_translated.{}".format(file_name,file_type), "w")
      fout.writelines(replacement)
      fout.close()  

  except IOError:
    print('No File Found')

def process_text(input_path):
  f = open(input_path,'a')
  f.write('\n\n\n')
  f.close()
  try:
    f = open(input_path,'r')
    #get type of file based on its name (name.type)
    file_name, file_type = os.path.splitext(input_path)
    file_type = file_type[1:].lower()

    #new text file to be written
    replacement = []

    #try:

    #if the file type contains comment or not (like .md or .txt files)
    #if the file is comment included, checks all possible comment formats 
    if comment_format_start.get(file_type) != None:
      comment_format_s = comment_format_start[file_type]
      #print(comment_format_s)
    else:
      comment_format_s =None

    #searches for ending comment character, if the file type supports
    if comment_format_end.get(file_type) != None :
      comment_format_e = comment_format_end[file_type]

    replacement = parse_text(file_type,comment_format_s,f,replacement)

    f.close()
    #write new text in a translated file
    if len(replacement) != 0:
      fout = open("{}_translated.{}".format(file_name,file_type), "w")
      fout.writelines(replacement)
      fout.close()  

  except IOError:
    print('No File Found')

path = input('Enter your file path, translated file will be saved in same directory: ')
process_text(path)

Enter your file path, translated file will be saved in same directory: validate.js


In [ ]:
'        """把文科和理科平均分合成一张excel电子表"""\n'.strip().endswith('"""')

True

In [ ]:
translate_text('"""把文科和理科平均分合成一张excel电子表"""'.split('"""')[1])

[{'translation_text': 'Synthesizing arts and science on average into an Excel spreadsheet.'}]